In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"
import findspark
findspark.init()
findspark.find()

'/content/spark-3.2.1-bin-hadoop3.2'

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
df = spark.read.csv("/content/drive/My Drive/house_full.csv")
df.printSchema()
df.show(20,truncate = 100)
df.count()
from pyspark.sql import functions as F
df.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in df.columns]).show()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)

+----------------------------------+---------------------------------------------------------------+--------------------------------------------+------+-------+-----------+
|                               _c0|                                                            _c1|                                         _c2|   _c3|    _c4|        _c5|
+----------------------------------+---------------------------------------------------------------+--------------------------------------------+------+-------+-----------+
|                             Title|                                                       Location|                                       Price|  Area|  Rooms|   Interior|
| Apartment Nieuwezijds Voorburgwal|            1012 RT Amsterdam (Burgwallen-Nieuwe Zijde) 

In [ ]:
from pyspark.sql.functions import col
df=df.filter(col("_c0")!="Title")
df.show(5, truncate =100)

+---------------------------------+---------------------------------------------------------------+--------------------------------------------+------+-------+-----------+
|                              _c0|                                                            _c1|                                         _c2|   _c3|    _c4|        _c5|
+---------------------------------+---------------------------------------------------------------+--------------------------------------------+------+-------+-----------+
|Apartment Nieuwezijds Voorburgwal|            1012 RT Amsterdam (Burgwallen-Nieuwe Zijde)        |                €5,000 per month            |180 m²|4 rooms|  Furnished|
|    Apartment Frederiksplein 40 B|                  1017 XN Amsterdam (De Weteringschans)        |                €3,400 per month            |125 m²|3 rooms|Upholstered|
|    Apartment George Gershwinlaan|                             1082 MT Amsterdam (Zuidas)        |                €3,000 per month         

In [ ]:
colomns = ['Title','Location','Price','Area','Rooms','Description']
df = df.toDF(*colomns)
df.show(20, truncate = 100)

+----------------------------------+---------------------------------------------------------------+--------------------------------------------+------+-------+-----------+
|                             Title|                                                       Location|                                       Price|  Area|  Rooms|Description|
+----------------------------------+---------------------------------------------------------------+--------------------------------------------+------+-------+-----------+
| Apartment Nieuwezijds Voorburgwal|            1012 RT Amsterdam (Burgwallen-Nieuwe Zijde)        |                €5,000 per month            |180 m²|4 rooms|  Furnished|
|     Apartment Frederiksplein 40 B|                  1017 XN Amsterdam (De Weteringschans)        |                €3,400 per month            |125 m²|3 rooms|Upholstered|
|     Apartment George Gershwinlaan|                             1082 MT Amsterdam (Zuidas)        |                €3,000 per month   

In [ ]:
from pyspark.sql.functions import split
 
df =df.withColumn("Title", split(col("Title"), " ").getItem(0))

df.show(20, truncate=100)

+---------+---------------------------------------------------------------+--------------------------------------------+------+-------+-----------+
|    Title|                                                       Location|                                       Price|  Area|  Rooms|Description|
+---------+---------------------------------------------------------------+--------------------------------------------+------+-------+-----------+
|Apartment|            1012 RT Amsterdam (Burgwallen-Nieuwe Zijde)        |                €5,000 per month            |180 m²|4 rooms|  Furnished|
|Apartment|                  1017 XN Amsterdam (De Weteringschans)        |                €3,400 per month            |125 m²|3 rooms|Upholstered|
|Apartment|                             1082 MT Amsterdam (Zuidas)        |                €3,000 per month            |126 m²|3 rooms|Upholstered|
|Apartment|                       1091 BS Amsterdam (Weesperzijde)        |                €3,450 per month     

In [ ]:
df = df.withColumn("Location", df.Location.substr(18,2))
df.show(5, truncate = 200)

+---------+--------+--------------------------------------------+------+-------+-----------+
|    Title|Location|                                       Price|  Area|  Rooms|Description|
+---------+--------+--------------------------------------------+------+-------+-----------+
|Apartment|      RT|                €5,000 per month            |180 m²|4 rooms|  Furnished|
|Apartment|      XN|                €3,400 per month            |125 m²|3 rooms|Upholstered|
|Apartment|      MT|                €3,000 per month            |126 m²|3 rooms|Upholstered|
|Apartment|      BS|                €3,450 per month            |150 m²|4 rooms|  Furnished|
|Apartment|      CR|                €1,500 per month            | 50 m²|2 rooms|  Furnished|
+---------+--------+--------------------------------------------+------+-------+-----------+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import when,regexp_replace
df = df.withColumn('Area',regexp_replace("Area",'m²','')
    )
df=df.withColumn("Area",df.Area.cast("int"))
df=df.withColumnRenamed('Area','Area(m²)')
df.show(20,truncate=100)
df.printSchema()


+---------+--------+--------------------------------------------+--------+-------+-----------+
|    Title|Location|                                       Price|Area(m²)|  Rooms|Description|
+---------+--------+--------------------------------------------+--------+-------+-----------+
|Apartment|      RT|                €5,000 per month            |     180|4 rooms|  Furnished|
|Apartment|      XN|                €3,400 per month            |     125|3 rooms|Upholstered|
|Apartment|      MT|                €3,000 per month            |     126|3 rooms|Upholstered|
|Apartment|      BS|                €3,450 per month            |     150|4 rooms|  Furnished|
|Apartment|      CR|                €1,500 per month            |      50|2 rooms|  Furnished|
|Apartment|      AB|                €1,450 per month            |      59|3 rooms|Upholstered|
|Apartment|      AB|                €1,425 per month            |      53|3 rooms|Upholstered|
|Apartment|      AB|                €1,450 per mon

In [ ]:
df = df.withColumn('Price',regexp_replace("Price",'€',''))
df = df.withColumn('Price',regexp_replace("Price",',',''))
df = df.withColumn('Price',regexp_replace("Price",'per',''))
df = df.withColumn('Price',regexp_replace("Price",'month',''))
df = df.withColumn('Price',df.Price.cast("int"))
df = df.withColumnRenamed('Price','Price(€)')
df.show(20,truncate=100)
df.printSchema()

+---------+--------+--------+--------+-------+-----------+
|    Title|Location|Price(€)|Area(m²)|  Rooms|Description|
+---------+--------+--------+--------+-------+-----------+
|Apartment|      RT|    5000|     180|4 rooms|  Furnished|
|Apartment|      XN|    3400|     125|3 rooms|Upholstered|
|Apartment|      MT|    3000|     126|3 rooms|Upholstered|
|Apartment|      BS|    3450|     150|4 rooms|  Furnished|
|Apartment|      CR|    1500|      50|2 rooms|  Furnished|
|Apartment|      AB|    1450|      59|3 rooms|Upholstered|
|Apartment|      AB|    1425|      53|3 rooms|Upholstered|
|Apartment|      AB|    1450|      59|3 rooms|Upholstered|
|Apartment|      AB|    1450|      59|3 rooms|Upholstered|
|Apartment|      AB|    1420|      52|3 rooms|Upholstered|
|Apartment|      TS|    1350|      60|2 rooms|Upholstered|
|Apartment|      GD|    1700|      86|4 rooms|       None|
|Apartment|      XC|    2150|      72|3 rooms|  Furnished|
|Apartment|      GS|    2200|      69|2 rooms|Upholstere

In [ ]:
df = df.withColumn('Rooms',regexp_replace("Rooms",'rooms',''))
df = df.withColumn('Rooms',regexp_replace("Rooms",'room',''))
df = df.withColumn('Rooms',df.Rooms.cast("int"))
df.show(20,truncate=100)
df.printSchema()

+---------+--------+--------+--------+-----+-----------+
|    Title|Location|Price(€)|Area(m²)|Rooms|Description|
+---------+--------+--------+--------+-----+-----------+
|Apartment|      RT|    5000|     180|    4|  Furnished|
|Apartment|      XN|    3400|     125|    3|Upholstered|
|Apartment|      MT|    3000|     126|    3|Upholstered|
|Apartment|      BS|    3450|     150|    4|  Furnished|
|Apartment|      CR|    1500|      50|    2|  Furnished|
|Apartment|      AB|    1450|      59|    3|Upholstered|
|Apartment|      AB|    1425|      53|    3|Upholstered|
|Apartment|      AB|    1450|      59|    3|Upholstered|
|Apartment|      AB|    1450|      59|    3|Upholstered|
|Apartment|      AB|    1420|      52|    3|Upholstered|
|Apartment|      TS|    1350|      60|    2|Upholstered|
|Apartment|      GD|    1700|      86|    4|       None|
|Apartment|      XC|    2150|      72|    3|  Furnished|
|Apartment|      GS|    2200|      69|    2|Upholstered|
|Apartment|      VM|    1400|  

In [ ]:
from pyspark.sql import functions as F
df.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in df.columns]).show()

+-----+--------+--------+--------+-----+-----------+
|Title|Location|Price(€)|Area(m²)|Rooms|Description|
+-----+--------+--------+--------+-----+-----------+
|    0|       0|       1|       0|    0|          0|
+-----+--------+--------+--------+-----+-----------+



The missing value of the price is because the webpage doesn't have the price

In [ ]:
df.write.csv("/content/drive/My Drive/Labs/house_clean")